In [87]:
using CSV, DataFrames
using FileIO, Dates
using DelimitedFiles

include("../region_cube_split/cubes.jl")
include("../region_network_create/network.jl")

add_properties (generic function with 1 method)

In [2]:
romania_full = CSV.read("../data/romania.csv", DataFrame);
romania = romania_full[romania_full.Datetime .> DateTime(1976,1,1,0,0,0),:];

vrancea = romania[(romania.Latitude .>= 45.20) .& (romania.Latitude .<= 46.0) .& 
                    (romania.Longitude .>= 26.0) .& (romania.Longitude .<= 27.0) .&
                    (romania.Depth .>= 50.0) .& (romania.Depth .<= 200.0), :];

In [3]:
vrancea, vrancea_cubes = region_cube_split(vrancea,side=5,energyRelease=true);

In [5]:
MG = create_network(vrancea, vrancea_cubes; edgeWeight=true)
# MG = add_properties(MG, vrancea_cubes)

{1727, 7612} undirected Int64 metagraph with Float64 weights defined by :weight (default weight 1.0)

In [141]:
# motif = "Triangle"
motif = "Tetrahedron"

"Tetrahedron"

In [143]:
filename= "./motifs" * motif * "_vrancea_5km.csv"

"./motifsTetrahedron_vrancea_5km.csv"

In [144]:
# motifs = CSV.read(filename, DataFrame;header=false)

# Creates a matrix: number of motifs x motif nodes (3 for triangles, 4 for tetrahedrons)
motifs = readdlm(filename, ',', Int64);
motifs;


In [145]:
# Calculate total energy and mean energy only for cubes that are part of motifs
# This is done to reduce calculations. I do not need to calculate these for each motif
# (because, nodes are repeated in motifs.)
# (I calculate separately, then I just add them up for each motif)

# Get all the cubes, uniquely
vec_motifs = unique!(vec(motifs));

# Will apend to a dictionary: cubeIndex => [totalEnergy, meanEnergy]
cube_energy = Dict()
for value in vec_motifs
    # Get the cube from df_cubes
    respective_cube = vrancea_cubes.cubeIndex[value]
    # Use the cube to get all the quakes in that cube from df
    quakesInNode = vrancea[vrancea.cubeIndex .== respective_cube,:];

    # Calculate total energy and mean energy
    totalEnergy = sum(quakesInNode.energyRelease)
    meanEnergy = totalEnergy / length(quakesInNode.energyRelease)

    # Push to dictionary
    cube_energy[value] = [totalEnergy, meanEnergy]
end

In [148]:
# Caclulate total energy and mean energy per each motif
motif_energy = Dict()
for i=1:length(motifs[:,1])
    # initialize with zero
    total_energy_in_motif=0
    mean_energy_in_motif=0
    # parse each motif (3 elements for Triangles, 4 for Tetrahedrons)
    for j=1:length(motifs[1,:])
        # add to total energy and mean energy
        total_energy_in_motif += cube_energy[motifs[i,j]][1] 
        mean_energy_in_motif += cube_energy[motifs[i,j]][2]  
    end
    # Add to dictionary
    motif_energy[i] = [total_energy_in_motif,mean_energy_in_motif]
end

In [149]:
motifs_energy

Dict{Any, Any} with 21977 entries:
  11950 => [1.54193e12, 7.31641e10]
  1703  => [4.8157e11, 2.06604e10]
  12427 => [1.63154e12, 5.0651e10]
  7685  => [9.99835e12, 4.06294e11]
  18374 => [5.59464e12, 1.61607e11]
  3406  => [2.09461e13, 1.38262e12]
  1090  => [2.45345e12, 9.88382e10]
  2015  => [5.7741e13, 1.96905e12]
  18139 => [2.48052e12, 1.06015e11]
  17088 => [5.40481e12, 1.65503e11]
  11280 => [2.69321e13, 1.53375e12]
  16805 => [2.61405e12, 1.0413e12]
  3220  => [2.51717e12, 1.00208e11]
  11251 => [2.19959e13, 1.43391e12]
  422   => [1.7971e13, 6.00658e11]
  15370 => [2.91777e15, 1.38809e14]
  15859 => [1.59227e12, 8.40716e10]
  4030  => [2.18481e12, 1.58372e11]
  8060  => [8.41971e11, 3.78217e10]
  ⋮     => ⋮

In [ ]:
using Geodesy

In [89]:
vrancea_cubes

Row,cubeIndex,xLatitude,yLongitude,zDepth,cubeLatitude,cubeLongitude,cubeDepth
,String,Int64,Int64,Int64,Float64,Float64,Float64
1,6433,8,6,23,45.5374,26.35,162.5
2,6183,10,8,22,45.6274,26.4773,157.5
3,5608,11,8,20,45.6724,26.4773,147.5
4,2514,13,12,9,45.7623,26.7318,92.5
5,5364,1,11,19,45.2225,26.6682,142.5
6,4077,10,3,15,45.6274,26.1591,122.5
7,4107,4,5,15,45.3575,26.2864,122.5
8,3632,15,10,13,45.8523,26.6046,112.5
9,1852,17,7,7,45.9423,26.4136,82.5


In [119]:
areas = Dict()
for i=1:length(motifs[:,1])
    x = Vector{Any}(undef,3)
    for j=1:3

        lat = vrancea_cubes.cubeLatitude[motifs[i,j]]
        lon = vrancea_cubes.cubeLongitude[motifs[i,j]]
        dep = vrancea_cubes.cubeDepth[motifs[i,j]]

        x[j] = LLA(lat,lon,-dep)

    end

    a = Geodesy.euclidean_distance(x[1],x[2]) / 1000
    b = Geodesy.euclidean_distance(x[2],x[3]) / 1000
    c = Geodesy.euclidean_distance(x[1],x[3]) / 1000;

    # calculate semiperimeter
    sp = (a+b+c)/2
            
    # Use Heron's formula Area = sqrt(semiperimeter(sp-a)(sp-b)(sp-c))
    # We have the areas of the triangle, append them to lists ! 

    A = sqrt(abs(sp*(sp-a)*(sp-b)*(sp-c)))

    areas[i] = A
end

In [126]:
filename="./motifsTetrahedron_vrancea_5km.csv"

"./motifsTetrahedron_vrancea_5km.csv"

In [128]:
# motifs = CSV.read(filename, DataFrame;header=false)

# Creates a matrix: number of motifs x motif nodes (3 for triangles, 4 for tetrahedrons)
motifs = readdlm(filename, ',', Int64);
motifs;


In [140]:
length(motifs[1,:])

4

### Volumes

In [130]:
volumes = Dict()
for i=1:length(motifs[:,1])
    x = Vector{Any}(undef,4)
    for j=1:4

        lat = vrancea_cubes.cubeLatitude[motifs[i,j]]
        lon = vrancea_cubes.cubeLongitude[motifs[i,j]]
        dep = vrancea_cubes.cubeDepth[motifs[i,j]]

        x[j] = LLA(lat,lon,-dep)

    end

    W = Geodesy.euclidean_distance(x[1],x[2]) / 1000
    V = Geodesy.euclidean_distance(x[2],x[3]) / 1000
    U = Geodesy.euclidean_distance(x[1],x[3]) / 1000
    u = Geodesy.euclidean_distance(x[2],x[4]) / 1000
    v = Geodesy.euclidean_distance(x[1],x[4]) / 1000
    w = Geodesy.euclidean_distance(x[3],x[4]) / 1000

    # calculate elements that go into elements that go into Heron formula
    A = (w-U+v)*(U+v+w)
    B = (u-V+w)*(V+w+u)
    C = (v-W+u)*(W+u+v)
    
    a = (U-v+w)*(v-w+U)
    b = (V-w+u)*(w-u+V)
    c = (W-u+v)*(u-v+W)

    # elements that go into Heron formula
    p = sqrt(abs(a*B*C))
    q = sqrt(abs(b*C*A))
    r = sqrt(abs(c*A*B))
    s = sqrt(abs(a*b*c))
            
    # Use Heron's formula Area = sqrt(semiperimeter(sp-a)(sp-b)(sp-c))
    # We have the areas of the triangle, append them to lists ! 

    V = sqrt(abs((-p+q+r+s)*(p-q+r+s)*(p+q-r+s)*(p+q+r-s)))/(192*u*v*w)

    volumes[i] = V
end

In [131]:
volumes

Dict{Any, Any} with 21977 entries:
  11950 => 1.023
  1703  => 1.07235
  12427 => 1.54003
  7685  => 0.225364
  18374 => 0.159408
  3406  => 0.247584
  1090  => 0.588097
  2015  => 0.54375
  18139 => 1.53502
  17088 => 0.174772
  11280 => 0.139509
  16805 => 1.13957
  3220  => 0.661264
  11251 => 0.614917
  422   => 0.206537
  15370 => 0.0909343
  15859 => 0.255473
  4030  => 0.231534
  8060  => 0.156393
  ⋮     => ⋮

In [136]:
# Define the function to do the regression with 
function power_law(x, a, b)
    return a*x^-b
end

power_law (generic function with 1 method)